# ML Model Example

This example provides a simply query tutorial for InfluxDB Cloud Dedicated. 

In [232]:
import influxdb_client_3 as InfluxDBClient3
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
import plotly.io as pio
import plotly.express as px
import matplotlib.pyplot as plt
pio.renderers.default = "vscode"

Next we provide our connection credentials to the InfluxDB 3.0 instance. 
`token` - scoped to the bucket you wish to query from.
`host` - the host of your cloud dedicated instance.
`client` - host wrapped in the flightclient class. 

In [233]:
token = ''
host = 'eu-central-1-1.aws.cloud2.influxdata.com'    
org = 'Jay-IOx'
db = 'factory'

client = InfluxDBClient3.InfluxDBClient3(token=token,
                         host=host,
                         org=org,
                         database=db) 
    
    


In [234]:
query = "SELECT vibration FROM machine_data WHERE \"machineID\" = 'machine1' AND time >= now() - 1h"
table = client.query(query=query, language="influxql")
normal_data = table.to_pandas()
print(normal_data)

     iox::measurement                          time  vibration
0        machine_data 2023-06-20 16:15:19.040721282       83.0
1        machine_data 2023-06-20 16:15:20.041552766       82.0
2        machine_data 2023-06-20 16:15:21.042487643       87.0
3        machine_data 2023-06-20 16:15:22.043313710       87.0
4        machine_data 2023-06-20 16:15:23.045128512       87.0
...               ...                           ...        ...
3585     machine_data 2023-06-20 17:15:07.256937964       86.0
3586     machine_data 2023-06-20 17:15:08.257660333       83.0
3587     machine_data 2023-06-20 17:15:09.258028704       87.0
3588     machine_data 2023-06-20 17:15:10.258651049       80.0
3589     machine_data 2023-06-20 17:15:11.260629025       84.0

[3590 rows x 3 columns]


In [235]:
fig = px.line(normal_data, x="time", y=["vibration"], title='raw vibration')
fig.show()

The `query_tools` function provides a wrapper to execution of a query agaisnt InfluxDB usinf flight. This function requires 3 paramters.
`database`- this specifies the database you wish to direct your query at.
`query` - This accepts a string literal of the query you wish to perform (SQL or InfluxQL).
`query_type` - Whether this is a SQL or InfluxQL based query.

Return all columns from the table flight for the past 3 days.

In [236]:
query = "SELECT vibration FROM machine_data WHERE \"machineID\" = 'machine2' AND time >= now() - 1h"
table = client.query(query=query, language="influxql")
anom_data = table.to_pandas()
fig2 = px.line(anom_data, x="time", y=["vibration"], title='raw vibration')
fig2.show()

In [237]:

# Feature Engineering: rolling mean and standard deviation
window_size = 10
normal_data['rolling_mean'] = normal_data['vibration'].rolling(window=window_size).mean()
normal_data['rolling_std'] = normal_data['vibration'].rolling(window=window_size).std()

anom_data['rolling_mean'] = normal_data['vibration'].rolling(window=window_size).mean()
anom_data['rolling_std'] = normal_data['vibration'].rolling(window=window_size).std()


ValueError: Length of values (3581) does not match length of index (3590)

In [ ]:
# Define the autoencoder model
model = Sequential([
    Dense(64, activation='relu', input_shape=(2,)),
    Dense(32, activation='relu'),
    Dense(64, activation='relu'),
    Dense(2)
])

model.compile(optimizer='adam', loss='mse')

# Train the autoencoder on data from machine 1 (healthy)
x_train = normal_data[['rolling_mean', 'rolling_std']].dropna().values
model.fit(x_train, x_train, epochs=100, batch_size=10)

Epoch 1/100
358/358 [==============================] - 1s 884us/step - loss: 133.4302
Epoch 2/100
358/358 [==============================] - 0s 869us/step - loss: 0.1411
Epoch 3/100
358/358 [==============================] - 0s 942us/step - loss: 0.1430
Epoch 4/100
358/358 [==============================] - 0s 863us/step - loss: 0.1374
Epoch 5/100
358/358 [==============================] - 1s 1ms/step - loss: 0.1345
Epoch 6/100
358/358 [==============================] - 0s 881us/step - loss: 0.1345
Epoch 7/100
358/358 [==============================] - 0s 845us/step - loss: 0.1273
Epoch 8/100
358/358 [==============================] - 0s 845us/step - loss: 0.1322
Epoch 9/100
358/358 [==============================] - 0s 847us/step - loss: 0.1239
Epoch 10/100
358/358 [==============================] - 0s 843us/step - loss: 0.1209
Epoch 11/100
358/358 [==============================] - 0s 856us/step - loss: 0.1079
Epoch 12/100
358/358 [==============================] - 0s 852us/step - lo

In [ ]:
# Use the trained autoencoder to reconstruct the anomalous data
reconstructed_data = model.predict()

# Calculate the reconstruction error (mean squared error)
mse = np.mean(np.power(anom_data - reconstructed_data, 2), axis=1)

# Set a threshold above which a high error would indicate an anomaly.
# Here, we are using the 99th percentile of the reconstruction error on the anomalous data as the threshold.
threshold = np.quantile(mse, 0.99)
print(f"Reconstruction error threshold: {threshold}")

# Detect anomalies. It's an array of booleans, True indicates anomalies
anomalies = mse > threshold
print(f"Number of anomalies detected: {np.sum(anomalies)}")

# Optionally, you can also add the anomaly flags to your original DataFrame for machine 3
anomalous_dataframe = anom_data.copy()
anomalous_dataframe['is_anomaly'] = anomalies

112/112 [==============================] - 0s 863us/step


KeyError: 'reconstruction_error'

In [ ]:
model.save('MLP.h5')